## This code should allow for analysis of TEM images of nuclei 

In [5]:
#Import packages for use
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import zipfile
import seaborn as sns
from PIL import Image, ImageDraw
import cv2
sns.set()
sns.set_style('white')

Define neccesary functions - from ARA

In [6]:
# Preprocessing.py

def to_gray(image, color_space="RGB", cv2_color=None):
    """
    Converts the inputted image to gray scale.

    This function should serve as a wrapper for cv2.cvtColor with basic input
    checking. Its main use is to convert color images to gray scale but it can
    be used as an alias to cv2.cvtColor if a color conversion code is passed
    into cv2_color

    Parameters
    ----------
    image : np.ndarray
        Image to convert to grayscale.

    color_space : str, optional
        Color space of the image. Valid entries are 'RGB', 'BGR'.

    cv2_color : cv2.ColorConversionCodes
        OpenCV color conversion code. Bypasses image array checks if used.

    Returns
    -------
    image_gray : np.ndarray
        The image converted to gray scale (ie len(image.shape) is 2)

    Raises
    ------
    InputError
        If input is not an image or color_space is invalid.
    """
    # Check if an OpenCV color conversion was entered
    if cv2_color is not None:
        color = cv2_color
        
    else:
        # Check we are getting an image
        if len(image.shape) < 2:
            raise InputError("Got array that is not an image. Shape is {0}."
                             "".format(image.shape))

        # Check that it isn't already grayscale
        if len(image.shape) < 3:
            logger.warning("Got image that is already grayscale.")
            return image

        # Check for the color space
        if color_space.upper() == "RGB":
            color = cv2.COLOR_RGB2GRAY
        elif color_space.upper() == "BGR":
            color = cv2.COLOR_BGR2GRAY
        else:
            raise InputError("Invalid color_space entry. Got '{0}'".format(
                color_space))

    # Do the conversion
    return cv2.cvtColor(image, color)

def to_uint8(image, mode="scale"):
    """
    *Correctly* converts an image to uint8 type.
    
    Running 'image.astype(np.uint8)' on its own applies a mod(256) to handle
    values over 256. The correct way is to either clip (implemented here) or
    normalize to the the max and min possible values of the array. Below are the
    available conversion modes:

    clip
        Truncates the  image at 0 and 255, then returns the resulting array.

    norm
        Normalizes the image so that the maximum value of the input array is set
        to 255 and the minimum value is 0.
    
    scale
        Scales the image so that the maximum and minimum values of the resulting
        array corresponds to the maximum and minimum possible values of the
        input array.

    Parameters
    ----------
    image : np.ndarray
        Image to be converted to uint8.

    mode : str, optional 
        Conversion mode to use. See conversion modes for more details.

    Returns
    -------
    np.ndarray
        Image that is a np.ndarray with dtype uint8.
    """
    # The main hurdle with performing the clipping is making sure we don't
    # create values that exceed the buffer size of the inputted dtype. For
    # example, for float32 the maximum value is 65504.0 and the min value is
    # 65504.0. If you try to find the range using a float32 by doing 65504.0 -
    # (-65504.0) you will get Inf.
    
    # Make sure the image is a numpy array
    if not isinstance(image, np.ndarray):
        image_array = np.array(image)
    else:
        image_array = np.copy(image)
        
    # Clip or normalize the image
    if mode.lower() == "clip":
        output = np.clip(image_array, 0, 255)

    # Normalize to max and min values of the array
    elif mode.lower() == "norm":
        range_pixels = image_array.max() - image_array.min() or 1        
        output = 255 * (image_array - image_array.min()) / range_pixels
        
    # Scale according to the max and min possible values of the array
    elif mode.lower() == "scale":
        try:
            # Grab array info for int types
            type_min = np.iinfo(image_array.dtype).min
            type_max = np.iinfo(image_array.dtype).max
        except ValueError:
            # Grab array info for float types
            type_min = np.finfo(image_array.dtype).min
            type_max = np.finfo(image_array.dtype).max
        # Use half the range to guarantee we can fit the range in the variable
        range_half = type_max/2 - type_min/2
        output = ((image_array/2 - type_min/2)/range_half) * 255

    # Handle invalid inputs
    else:
        raise InputError("Invalid conversion mode inputted. Valid modes are " 
                         "'clip' and 'norm.'")

    # Warn the user if the preprocessing resulted in a zeroed array
    if not output.any() and image_array.any():
        logger.warning("to_uint8 resulted in a fully zeroed array from "
                       "non-zero input.")

    # Return casting as uint8
    return output.astype(np.uint8)

def uint_resize_gauss(image, mode='scale', fx=1.0, fy=1.0, kernel=(11,11), 
                      sigma=0):
    """
    Preprocess the image by converting to uint8, resizing and running a 
    gaussian blur.
    
    Parameters
    ----------
    image : np.ndarray
        The image to be preprocessed.

    mode : str, optional 
        Conversion mode that either clips the image or normalizes to the range
        of the original image type.

    fx : float, optional
        Percent to resize the image by in x.

    fy : float, optional
        Percent to resize the image by in y.

    kernel : tuple, optional
        Kernel to use when running the gaussian filter.

    Returns
    -------
    np.ndarray 
        Image of type uint8 that has been resized and blurred.
    """
    image_uint = to_uint8(image, mode=mode)
    image_resized = cv2.resize(image_uint, (0,0), fx=fx, fy=fy)
    image_gblur = cv2.GaussianBlur(image_resized, kernel, sigma)
    return image_gblur

def threshold_image(image, binary=True, mode="top", factor=1, **kwargs):
    """
    Thresholds the image, creating a binary image.

    The thresholding can be done using one of the following modes:

    mean
        Set the threshold line to be image.mean + image.std*factor.

    top
        Sets the threshold line to be image.max - image.std*factor, leaving just
        the highest intensity pixels.

    bottom
        Sets the threshold line to be image.min + image.std*factor, removing
        just the lowest intensity pixels

    adaptive
        Sets threshold line according to a weighed sum of neughborhood values
        using a gaussian window. See 'Adaptive Thresholding' in the following
        link for more details.
        http://docs.opencv.org/trunk/d7/d4d/tutorial_py_thresholding.html

    otsu
        Sets the threshold to be between the histogram peaks of a bimodal image.
        See "Otsu's Binarization" in the following for more details.
        http://docs.opencv.org/trunk/d7/d4d/tutorial_py_thresholding.html

    Parameters
    ----------
    image : np.ndarray
        The image to threshold.

    binary : bool, optional
        Use binary thresholding or to_zero thresholding.

    mode : str, optional
        Thresholding mode to use. See docstring for more information.

    factor : int, optional
        Number of times to multiply the std by before adding to the mean for
        thresholding.    
        
    Returns
    -------
    th : np.ndarray
        Image that has been thresholded.
    """
    valid_modes = set(['mean', 'top', 'bottom', 'adaptive', 'otsu'])
    if binary:
        th_type = cv2.THRESH_BINARY
    else:
        th_type = cv2.THRESH_TOZERO

    # Find the right mode
    if mode.lower() not in valid_modes:
        raise InputError("Invalid mode passed for thresholding.")
    elif mode.lower() == 'mean':
        _, th = cv2.threshold(image, image.mean() + image.std()*factor, 
                              255,
                              th_type)        
    elif mode.lower() == 'top':
        _, th = cv2.threshold(image, image.max() - image.std()*factor, 
                              255,
                              th_type)
    elif mode.lower() == 'bottom':
        _, th = cv2.threshold(image, image.min() + image.std()*factor, 
                              255,
                              th_type)
    elif mode.lower() == "adaptive":
        blockSize = kwargs.pop("BlockSize", 11)
        C = kwargs.pop("C", 2)        
        th = cv2.adaptiveThreshold(image, 
                                   255, 
                                   cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   th_type, 
                                   blockSize, 
                                   C)
    elif mode.lower() == "otsu":
        _, th = cv2.threshold(image, 0, 255, th_type+cv2.THRESH_OTSU)

    return th



def get_opening(image, n_erode=1, n_dilate=1, kernel=(5,5)):
    """
    Performs the specified number of erosions, followed by the specified number
    of dilations to get the opening of the image.

    Parameters
    ----------
    image : np.ndarray
        The image to perform the opening on. Must be a binary image

    n_erode : int, optional
        The number of times to perform an erosion on the image.

    n_dilate : int, optional
        The number of times to perform a dilation on the image.

    kernel : tuple, optional
        The kernel size to use when eroding and dilating.

    Returns
    -------
    image_opened : np.ndarray
        Image that has had n_erode erosions followed by n_dilate dilations.

    Raises
    ------
    InputError
        When image passed is not a binary image
    """
    # Check that image is binary
    if len(np.unique(image)) > 2:
        raise InputError("Binary image is required for morphological "
                         "transformations.")
    kernel = np.ones(kernel, dtype=np.uint8)    
    image_eroded = cv2.erode(image, kernel, iterations=n_erode)
    image_opened = cv2.dilate(image_eroded, kernel, iterations=n_dilate)
    return image_opened

def get_closing(image, n_erode=1, n_dilate=1, kernel=(5,5)):
    """
    Performs the specified number of dilations, followed by the specified number
    of erosions to get the closing of the image.

    Parameters
    ----------
    image : np.ndarray
        The image to perform the closing on.

    n_erode : int, optional
        The number of times to perform an erosion on the image.

    n_dilate : int, optional
        The number of times to perform a dilation on the image.

    kernel : tuple, optional
        The kernel size to use when eroding and dilating.

    Returns
    -------
    image_opened : np.ndarray
        Image that has had n_dilate dilations followed by n_erode erosions.

    Raises
    ------
    InputError
        When image passed is not a binary image
    """
    if len(np.unique(image)) > 2:
        raise InputError("Binary image is required for morphological "
                         "transformations.")
    kernel = np.ones(kernel, dtype=np.uint8)
    image_dilated = cv2.dilate(image, kernel, iterations=n_dilate)
    image_closed = cv2.erode(image_dilated, kernel, iterations=n_erode)
    return image_closed

Obtain your nucleus outline that was drawn from FIJI

In [1]:
def line_array(cell_path):
#Code to turn NPerimeter line into X,Y coordinates
    img_conversion = 293 

    #Define our path as anything in folders with an .txt in it    
    assert cell_path.exists()

    NP_txt = cell_path / Path(cell_path.name + '_NPerimeter.txt')
    print(NP_txt)
    assert NP_txt.exists()

    #Read the X,Y coordinates from the NP.txt file and convert to pixels
    df = np.round(pd.read_table(str(NP_txt), header=None)*img_conversion, decimals=0)
    df_diff = df.diff()
    
    #Finding Euclidean distance
    df_ed = np.sqrt(df_diff.iloc[:,0]**2 +df_diff.iloc[:,1]**2) 
        #This code will only give you some pixel X,Y coordinates, not everything on that line

    
    #Get the distances of all X,Y coordinates on that line:
    #If the distance between 2 pixel points is more than 1, fill in the blanks by 1 to make a continuous line
    x_fin, y_fin = [], [] 
    for i in range(len(df)): 
        x, y = df.iloc[i,:]
        x_fin.append(x)
        y_fin.append(y)
    
    #Create an array for all X,Y coordinates along your NP line
    L = np.array([x_fin,y_fin]).T    

    #Self check: If you want to see all of your X,Y values of your line now
   # Ltabletest = {'X values':x_fin, 'Y values':y_fin}
    #Ltable_pd = pd.DataFrame(Ltabletest)
    #Ltable_pd.head()
    
    return L

In [9]:
#Thresholding
def threshold(cell_path):
#Get image, turn into array, confirm single channel from shape
    nuc_image = Image.open(str(cell_path/ cell_path.stem) + '.TIF')

# Turn into a np array, resize 16-bit to 8-bit
    nuc_image_np = to_uint8(np.array(nuc_image))
#     nuc_thresh = np.invert(threshold_image(nuc_image_np, mode='otsu'))
# Contrast enhancement    
    clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(21,21))
    nuc_image_clahe = clahe.apply(nuc_image_np)
    
 # Otsu adaptive thresholding on contrast enhanced image
    nuc_clahe_thresh = threshold_image(nuc_image_clahe, mode='otsu')
    nuc_clahe_inv = np.invert(nuc_clahe_thresh)
    return nuc_clahe_inv

In [10]:
def create_mask(thresh_img, L, cell_path):
    img = Image.new('L', thresh_img.shape[::-1], 0)
    polygon = [tuple(x) for x in L]
    ImageDraw.Draw(img).polygon(polygon, outline=1, fill=1)
    mask = np.array(img)
    return mask

In [11]:
# cell_path = Path('C:/Users/audre/Documents/Python Scripts/Aizenman Lab/Nuclei Analysis/Cell Files/Control/Cell20')
Chromatin_percent= {}

Cell_files = Path('Cell Files/')
assert Cell_files.exists()
for condition in Cell_files.iterdir():
    for Cellfolders in condition.iterdir():
        cell_img = Cellfolders / Path(Cellfolders.name + '.TIF')
        if not cell_img.exists(): 
            print('{} Cell file doesnt exist'.format(cell_img))
            continue
        
        nuc_outline = Cellfolders / Path(Cellfolders.name + '_NPerimeter.txt')
        if not nuc_outline.exists():
            print('{} Nuc file doesnt exist'.format(nuc_outline))
            continue
        
#        #Save the four data file ('L.txt') in it's correct folders
#         out_file_names = ['L.txt']
#         if not np.all([Path(Cellfolders / out_file).exists() for out_file in out_file_names]):
            
#        #Call our line function to create lines for each Cellfolder
        nucleus = line_array(Cellfolders)
#             for arr, name in zip([nucleus], out_file_names):
#                 nuc_ret_path = Path(Synfolders / name)
#                 if not nuc_ret_path.exists():
#                     np.savetxt(str(nuc_ret_path), arr)
            
        thresh_img = threshold(Cellfolders)
        mask = create_mask(thresh_img, nucleus, Cellfolders)

        masked_image = np.multiply(thresh_img, mask)
        mask_area = np.sum(mask)

        sum_pixels = np.sum(masked_image/masked_image.max())
        percent_chrom = sum_pixels/mask_area
        
        Chromatin_percent[Cellfolders.stem]=percent_chrom

#      #Shows us progress while it's working, lets us know when it has completed for each condition             
#             print('Completed {0}!'.format(Cellfolders))
#         else:
#             print('Found all distance files, skipping {0}...'.format(Cellfolders))
    print('Completed cells for condition {0}'.format(condition.name))

Cell Files\Control\Cell103\Cell103_NPerimeter.txt
Cell Files\Control\Cell104\Cell104.TIF Cell file doesnt exist
Cell Files\Control\Cell110\Cell110.TIF Cell file doesnt exist
Cell Files\Control\Cell111\Cell111.TIF Cell file doesnt exist
Cell Files\Control\Cell112\Cell112.TIF Cell file doesnt exist
Cell Files\Control\Cell114\Cell114.TIF Cell file doesnt exist
Cell Files\Control\Cell125\Cell125.TIF Cell file doesnt exist
Cell Files\Control\Cell126\Cell126.TIF Cell file doesnt exist
Cell Files\Control\Cell127\Cell127.TIF Cell file doesnt exist
Cell Files\Control\Cell128\Cell128.TIF Cell file doesnt exist
Cell Files\Control\Cell19\Cell19_NPerimeter.txt
Cell Files\Control\Cell20\Cell20_NPerimeter.txt
Cell Files\Control\Cell21\Cell21_NPerimeter.txt
Cell Files\Control\Cell22\Cell22_NPerimeter.txt
Cell Files\Control\Cell23\Cell23_NPerimeter.txt
Cell Files\Control\Cell24\Cell24_NPerimeter.txt
Cell Files\Control\Cell25\Cell25_NPerimeter.txt
Cell Files\Control\Cell29\Cell29_NPerimeter.txt
Cell Fil

In [12]:
print(Chromatin_percent)

{'Cell103': 0.626545726752535, 'Cell19': 0.5297422087591942, 'Cell20': 0.5682909284169858, 'Cell21': 0.5100508515067745, 'Cell22': 0.543799786782757, 'Cell23': 0.5932435983532366, 'Cell24': 0.53460939858966, 'Cell25': 0.60929815763709, 'Cell29': 0.5201085186059647, 'Cell30': 0.6683793038277246}
